In [1]:
import os
import cv2
import mediapipe as mp
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

import feedforward

## Setup folder and register the training data

In [2]:
DATA_PATH = os.path.join('dist') 

In [3]:
feedforward.create_folder_if_not_exists(DATA_PATH)

In [4]:
feedforward.clear_folder(DATA_PATH)

In [6]:
num_videos = 20
num_frames_per_video = 100
prefix = 'arrow_wrist'

In [7]:
feedforward.register_videos(DATA_PATH, prefix, num_videos, num_frames_per_video)

2023-10-12 22:28:51.945 python[48295:6604807] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 

## Build Model

In [8]:
prefix_for_augmented = "augmented_"
feedforward.augment_dataset(DATA_PATH, prefix_for_augmented)

Processing video #0 / 20...


OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


Processing video #1 / 20...


OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


Processing video #2 / 20...


...

OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


Processing video #18 / 20...


OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


Processing video #19 / 20...


OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


In [ ]:
feedforward.delete_files_with_prefix(DATA_PATH, prefix_for_augmented)

In [9]:
window_len = 20
X_training, y_training = feedforward.build_training_data(DATA_PATH, window_len)
X_training.shape, y_training.shape

Processing video #1...


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Processing video #2...
Processing video #3...
Processing video #4...
Processing video #5...
Processing video #6...
Processing video #7...
Processing video #8...
Processing video #9...
Processing video #10...
Processing video #11...
Processing video #12...
Processing video #13...
Processing video #14...
Processing video #15...
Processing video #16...
Processing video #17...
Processing video #18...
Processing video #19...
Processing video #20...
Processing video #21...
Processing video #22...
Processing video #23...
Processing video #24...
Processing video #25...
Processing video #26...
Processing video #27...
Processing video #28...
Processing video #29...
Processing video #30...
Processing video #31...
Processing video #32...
Processing video #33...
Processing video #34...
Processing video #35...
Processing video #36...
Processing video #37...
Processing video #38...
Processing video #39...
Processing video #40...


((3231, 10, 2), (3231, 2))

In [10]:
model = Sequential()

model.add(LSTM(64, input_shape=(X_training.shape[1], X_training.shape[2]), activation='relu', return_sequences=True)) # LSTM layer with input shape (X_training.shape[1], X_training.shape[2]))
model.add(LSTM(64, activation='relu', return_sequences=False)) # LSTM layer with return_sequences=False for the final prediction# LSTM layer with return_sequences=False for the final prediction
model.add(Dense(2))# Dense output layer with N units

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

history = model.fit(X_training, y_training, batch_size=32, epochs=100, validation_split=0.2)

model.summary()

Epoch 1/100
81/81 [==============================] - 2s 7ms/step - loss: 0.0864 - accuracy: 0.7121 - val_loss: 0.0264 - val_accuracy: 0.7187
Epoch 2/100
81/81 [==============================] - 0s 5ms/step - loss: 0.0222 - accuracy: 0.7519 - val_loss: 0.0172 - val_accuracy: 0.7682
Epoch 3/100
81/81 [==============================] - 0s 5ms/step - loss: 0.0172 - accuracy: 0.7728 - val_loss: 0.0149 - val_accuracy: 0.7805
Epoch 4/100
81/81 [==============================] - 0s 5ms/step - loss: 0.0137 - accuracy: 0.8038 - val_loss: 0.0126 - val_accuracy: 0.8192
Epoch 5/100
81/81 [==============================] - 0s 5ms/step - loss: 0.0118 - accuracy: 0.8437 - val_loss: 0.0126 - val_accuracy: 0.8454
Epoch 6/100
81/81 [==============================] - 0s 5ms/step - loss: 0.0110 - accuracy: 0.8618 - val_loss: 0.0097 - val_accuracy: 0.8594
Epoch 7/100
81/81 [==============================] - 0s 6ms/step - loss: 0.0106 - accuracy: 0.8653 - val_loss: 0.0105 - val_accuracy: 0.8640
Epoch 8/100
8

## Test

In [11]:
cap = cv2.VideoCapture(feedforward.CAMERA_INDEX)
holistic = mp.solutions.holistic.Holistic()
frame_buffer = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = holistic.process(frame_rgb)

    if results.pose_landmarks:
        right_wrist = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.RIGHT_WRIST]

        height, width, _ = frame.shape

        wrist_x, wrist_y = int(right_wrist.x * width), int(right_wrist.y * height)

        cv2.circle(frame, (wrist_x, wrist_y), 5, (255, 0, 0), -1)

        wrist_coords = [
            right_wrist.x,
            right_wrist.y,
            # right_wrist.z
        ]

        frame_buffer.append(wrist_coords)

        if len(frame_buffer) >= window_len/2:
            prediction_input = np.array(frame_buffer)[np.newaxis, :, :]
            prediction = model.predict(prediction_input)

            prediction_x = int(prediction[0][0] * width)
            prediction_y = int(prediction[0][1] * height)

            cv2.circle(frame, (prediction_x, prediction_y), 5, (255, 0, 255), -1)
            
            cv2.arrowedLine(frame, (wrist_x, wrist_y), (prediction_x, prediction_y), (255, 0, 255), 2)

            frame_buffer = frame_buffer[1:]

    cv2.imshow('Video', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 10ms/step
